# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [4]:
# your code here
orders = pd.read_csv('Orders.zip')
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [5]:
# your code here
aggregated = orders.groupby('CustomerID').agg({'amount_spent': 'sum'})
aggregated

,amount_spent
CustomerID,
12346,77183.60
12347,4310.00
12348,1797.24
12349,1757.55
12350,334.40
...,...
18280,180.60
18281,80.82
18282,178.05


In [6]:
quantiles = aggregated.quantile([0.25, 0.75])
selected = aggregated[(aggregated['amount_spent'] >= quantiles.iloc[0]['amount_spent']) & (aggregated['amount_spent'] <= quantiles.iloc[1]['amount_spent'])]
print(selected)

            amount_spent
CustomerID              
12350             334.40
12354            1079.40
12355             459.40
12358            1168.06
12363             552.00
...                  ...
18252             526.67
18261             324.24
18263            1213.16
18265             801.51
18276             335.86

[2169 rows x 1 columns]


In [8]:
total_spent = orders.groupby('CustomerID')['amount_spent'].sum()


In [9]:
vip_threshold = total_spent.quantile(0.95) # top 5% of customers
preferred_threshold = total_spent.quantile(0.75) # top 25% of customers


In [14]:
orders['CustomerLabel'] = 'Regular'
orders.loc[orders['amount_spent'] >= vip_threshold, 'CustomerLabel'] = 'VIP'
orders.loc[(orders['amount_spent'] >= preferred_threshold) & (orders['amount_spent'] < vip_threshold), 'CustomerLabel'] = 'Preferred'
orders['CustomerLabel'].value_counts()

Regular      397792
Preferred       125
VIP               7
Name: CustomerLabel, dtype: int64

Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [16]:
# your code here
vip_customers = orders[orders['CustomerLabel'] == 'VIP']


In [17]:
vip_customers_by_country = vip_customers.groupby('Country')['CustomerID'].nunique()

In [18]:
vip_customers_by_country = vip_customers_by_country.sort_values(ascending=False)

In [19]:
print('The country with the most VIP customers is:', vip_customers_by_country.index[0])

The country with the most VIP customers is: United Kingdom


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [20]:
# your code here

vip_preferred_customers = orders[(orders['CustomerLabel'] == 'VIP') | (orders['CustomerLabel'] == 'Preferred')]

In [21]:
vip_preferred_customers_by_country = vip_preferred_customers.groupby('Country')['CustomerID'].nunique()

In [22]:
ip_preferred_customers_by_country = vip_preferred_customers_by_country.sort_values(ascending=False)

In [23]:
print('The country with the most VIP+Preferred customers is:', vip_preferred_customers_by_country.index[0])

The country with the most VIP+Preferred customers is: Australia
